## Coarse Graining Polymers using GRiTS

In [6]:
import grits
import mbuild as mb
import inspect
from polymer_dictionary import bond_indices
from polymer_dictionary import polymer_fragments
from polymer_dictionary import element_dict
import gsd
import os
import glob
import string
from grits import CG_System

In [13]:
key_list = sorted(list(bond_indices.keys())) # Aligning dictionary and path to aid automation
path = (os.getcwd()+"/") # Non-user specific path
molecule_list = sorted(glob.glob(path+"/mol2/"+"*.mol2"))
molecule_list.pop(10) # Removing polymer not tested
print(key_list)

['PCPDTFBT_C11_BO', 'PCPDTFBT_C1_BO', 'PCPDTFBT_C3_BO', 'PCPDTFBT_C4_BO', 'PCPDTFBT_C5_BO', 'PCPDTPT_HD', 'PCPDTPT_ODD', 'PCPDTPT_eneODD', 'PCPDTPT_nC16', 'PCPDT_PT_eneHD', 'PIDTBT_nC16', 'PIDTCPDT_C11BO', 'PIDTFBT_C11_BO']


In [10]:
# Function to parse through GSD nomenclature and automatically get backbone segments
def cg(filename):
    letters = string.ascii_uppercase
    name = os.path.basename(filename)
    name = (os.path.splitext(name)[0])[1:]
    new_gsd_name = "" + name
    
    final_cg = [polymer_fragments["P"]]
    grits_string = {}
    if name[0:4] == "CPDT":
        final_cg.append(polymer_fragments["CPDT"])
        name = name[4:]
    if name[0:3] == "IDT":
        final_cg.append(polymer_fragments["IDT"])
        name = name[3:]
    if name[0:3] == "FBT":
        final_cg.append(polymer_fragments["FBT"])
        name = name[3:]
    if name[0:2] == "PT":
        final_cg.append(polymer_fragments["PT"])
        name = name[2:]
    if name[0:2] == "_PT":
        final_cg.append(polymer_fragments["PT"])
        name = name[2:]
    if name[0:4] == "CPDT":
        final_cg.append(polymer_fragments["CPDT"])
        name = name[4:]
    if name[0:10] == "CPDT_eneHD":
        final_cg.append(polymer_fragments["CPDT_eneHD"])
        name = name[10:0]
    for i in range(len(final_cg)):
        key = ("_"+letters[i]) # Formatting for GRiTS into dictionary with a letter as the key
        grits_string[key] = final_cg[i] # SMIILES string of the backbone segment as the value
    system = CG_System(
        filename,
        beads=grits_string,
        conversion_dict=element_dict
    )
    cg_gsd = "CG-" + new_gsd_name + ".gsd"
    system.save(cg_gsd)
    return(grits_string)


In [11]:
backbones = cg(path+"gsd/10_mers/PCPDTFBT_C4_BO_10mer_linear_test.gsd")

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/grits/coarsegrain.py:153: UserWarning: c1ccncc1 not found in compound!
  warn(f"{smart_str} not found in compound!")
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/grits/coarsegrain.py:153: UserWarning: c1(F)ccc2nsnc2c1 not found in compound!
  warn(f"{smart_str} not found in compound!")


In [14]:
gsdfile = path+"alternate_PCPDTPT_nC16_melt.gsd"
system = CG_System(
    gsdfile,
    beads={"_A": "C1SC2C3SC(C4C5=NSN=C5C=NC=4)=CC=3C(CCCCCCCCCCCCCCCC)(CCCCCCCCCCCCCCCC)C=2C=1"},
    conversion_dict=element_dict
)

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/grits/coarsegrain.py:153: UserWarning: C1SC2C3SC(C4C5=NSN=C5C=NC=4)=CC=3C(CCCCCCCCCCCCCCCC)(CCCCCCCCCCCCCCCC)C=2C=1 not found in compound!
  warn(f"{smart_str} not found in compound!")
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/grits/coarsegrain.py:194: UserWarning: Some atoms have been left out of coarse-graining!
  warn("Some atoms have been left out of coarse-graining!")


In [4]:
cg_gsd = "cg_test.gsd"
system.save(cg_gsd)

ValueError: need at least one array to concatenate

In [7]:
# Appending a frame of a linear chain
original_traj = gsd.hoomd.open(path+"gsd_files/10_mers/PCPDTFBT_C4_BO_melt.gsd","r+")
new_traj = gsd.hoomd.open(path+"PCPDTFBT_C4_BO_10mer_linear.gsd","r+")
for frame in original_traj:
    new_traj.append(frame)

101
103
103
